**Initialize database**

In [223]:

/* Drop Tables */

set role postgres;

DROP TABLE IF EXISTS accidents;
DROP TABLE IF EXISTS archive;
DROP TABLE IF EXISTS manager_on_contract;
DROP TABLE IF EXISTS raw_contracts;
DROP TABLE IF EXISTS cars;
DROP TABLE IF EXISTS clients;
DROP TABLE IF EXISTS managers;
DROP TABLE IF EXISTS parkings;
DROP TABLE IF EXISTS users;




/* Create Tables */

CREATE TABLE accidents
(
	ID_accident int NOT NULL UNIQUE,
	vin_number varchar(100) NOT NULL UNIQUE,
	PRIMARY KEY (ID_accident)
) WITHOUT OIDS;


CREATE TABLE archive
(
	ID_record serial UNIQUE,

	ID_contract int NOT NULL,
	start_date date NOT NULL,
	end_date date NOT NULL,
	total_price money NOT NULL,

	car_vin_number varchar(100) NOT NULL UNIQUE,
	
	client_driving_license varchar(10) NOT NULL UNIQUE,
	client_name varchar(100) NOT NULL,
	phone_number varchar(20) UNIQUE,

	manager_passport_details varchar(90) NOT NULL UNIQUE,
	manager_name varchar(100) NOT NULL,

	PRIMARY KEY (ID_record)
) WITHOUT OIDS;


CREATE TABLE cars
(
	vin_number varchar(100) NOT NULL UNIQUE,
	cost_per_hour money,
	car_brand varchar(40) NOT NULL,
	car_color varchar(20) NOT NULL,
	car_class varchar(20) NOT NULL,
	transmission_type varchar(20) NOT NULL,
	fuel_type varchar(20) NOT NULL,
	ID_parking int NOT NULL,
	PRIMARY KEY (vin_number)
) WITHOUT OIDS;


CREATE TABLE clients
(
	driving_license varchar(10) NOT NULL,
	-- Фамилия Имя Отчество клиента
	name varchar(100) NOT NULL,
	-- Потребуется для быстрой связи с клиентом. Тип данных - могло быть и число, но для простоты учебного примера - пусть будет строка
	phone varchar(20) UNIQUE,
	-- Электронный адрес
	email varchar(50) UNIQUE,
	date_of_birth date,
	registration_address varchar,
	residence_address varchar,
	username varchar(20) NOT NULL UNIQUE,
	PRIMARY KEY (driving_license)
) WITHOUT OIDS;


CREATE TABLE managers
(
	manager_passport_details varchar(90) NOT NULL UNIQUE,
	name varchar(100) NOT NULL,
	phone varchar(20) NOT NULL UNIQUE,
	expirience smallint DEFAULT 0,
	salary int,
	username varchar(20) NOT NULL UNIQUE,
	ID_parking int NOT NULL UNIQUE,
	PRIMARY KEY (manager_passport_details)
) WITHOUT OIDS;


CREATE TABLE manager_on_contract
(
	manager_passport_details varchar(90) NOT NULL UNIQUE,
	ID_contract int NOT NULL UNIQUE
) WITHOUT OIDS;


CREATE TABLE parkings
(
	ID_parking int NOT NULL UNIQUE,
	address varchar(100) NOT NULL,
	PRIMARY KEY (ID_parking)
) WITHOUT OIDS;


CREATE TABLE raw_contracts
(
	ID_contract int NOT NULL UNIQUE,
	rental_duration interval NOT NULL,
	total_price money NOT NULL,
	driving_license varchar(10) NOT NULL UNIQUE,
	vin_number varchar(100) NOT NULL UNIQUE,
	signed boolean DEFAULT 'FALSE' NOT NULL,
	PRIMARY KEY (ID_contract)
) WITHOUT OIDS;


CREATE TABLE users
(
	username varchar(20) NOT NULL UNIQUE,
	password varchar(30) NOT NULL,
	role varchar(20) NOT NULL,
	PRIMARY KEY (username)
) WITHOUT OIDS;



/* Create Foreign Keys */

ALTER TABLE accidents
	ADD FOREIGN KEY (vin_number)
	REFERENCES cars (vin_number)
;


ALTER TABLE raw_contracts
	ADD FOREIGN KEY (vin_number)
	REFERENCES cars (vin_number)
;


ALTER TABLE raw_contracts
	ADD FOREIGN KEY (driving_license)
	REFERENCES clients (driving_license)
;


ALTER TABLE manager_on_contract
	ADD FOREIGN KEY (manager_passport_details)
	REFERENCES managers (manager_passport_details)
;


ALTER TABLE cars
	ADD FOREIGN KEY (ID_parking)
	REFERENCES parkings (ID_parking)
;


ALTER TABLE managers
	ADD FOREIGN KEY (ID_parking)
	REFERENCES parkings (ID_parking)
;


ALTER TABLE manager_on_contract
	ADD FOREIGN KEY (ID_contract)
	REFERENCES raw_contracts (ID_contract)
	ON UPDATE CASCADE
	ON DELETE CASCADE
;


ALTER TABLE clients
	ADD FOREIGN KEY (username)
	REFERENCES users (username)
;


ALTER TABLE managers
	ADD FOREIGN KEY (username)
	REFERENCES users (username)
;



/* Comments */

COMMENT ON COLUMN clients.name IS 'Фамилия Имя Отчество клиента';
COMMENT ON COLUMN clients.phone IS 'Потребуется для быстрой связи с клиентом. Тип данных - могло быть и число, но для простоты учебного примера - пусть будет строка';
COMMENT ON COLUMN clients.email IS 'Электронный адрес';

SET

DROP TABLE

DROP TABLE

DROP TABLE

DROP TABLE

...

ALTER TABLE

COMMENT

COMMENT

COMMENT

Total execution time: 00:00:00.245

**Temp filling**

In [224]:
delete from users;

INSERT INTO users (username, password, role) VALUES
('user1', 'password1', 'client'),
('user2', 'password2', 'manager'),
('user3', 'password3', 'admin'),
('user4', 'password4', 'manager');

SELECT * FROM users;

DELETE 0

INSERT 0 4

(4 row(s) affected)

Total execution time: 00:00:00.012

username,password,role
user1,password1,client
user2,password2,manager
user3,password3,admin
user4,password4,manager


**Functions**

In [225]:
drop function if exists check_auth;
drop function if exists archive_contract;
drop function if exists add_new_user_role;

create function check_auth(input_username varchar, input_password varchar)
returns varchar as 
$$
declare 
    user_role varchar;
begin
    select role into user_role from users u
    where input_username = u.username and input_password = u.password;
    return user_role;
end
$$ LANGUAGE plpgsql;


create function archive_contract() returns trigger as $$
begin
    if new.signed = true then
        insert into archive ( ID_contract, 
            start_date, 
            end_date, 
            total_price, 
            car_vin_number, 
            client_driving_license, 
            client_name, 
            phone_number, 
            manager_passport_details, 
            manager_name) values (
                new.ID_contract,
                now(),
                now() + new.rental_duration,
                new.total_price,
                new.vin_number,
                new.driving_license,
                (select name from clients c where c.driving_license = new.driving_license),
                (select phone from clients c where c.driving_license = new.driving_license),
                (select manager_passport_details from managers m where m.manager_passport_details in (
                    select manager_passport_details from manager_on_contract mc where mc.ID_contract = new.ID_contract)
                ),
                (select name from managers m where m.manager_passport_details in (
                    select manager_passport_details from manager_on_contract mc where mc.ID_contract = new.ID_contract)
                )
        );

        delete from raw_contracts where new.ID_contract = old.ID_contract;
        return new;
    end if;
end;
$$ language plpgsql;

DROP FUNCTION

DROP FUNCTION

NOTICE: function add_new_user_role() does not exist, skipping

DROP FUNCTION

CREATE FUNCTION

CREATE FUNCTION

Total execution time: 00:00:00.033

**Test**

In [226]:
select 'user1' as user, check_auth('user1', 'password1') as user_role;
select 'user3' as user, check_auth('user3', 'password3') as user_role;
select 'new user' as user, check_auth('new user', 'incorrect_password') as user_role;

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.018

user,user_role
user1,client


user,user_role
user3,admin


user,user_role
new user,NULL


**Procedures**

In [227]:
drop procedure if exists add_user;

create procedure add_user (user_name varchar, user_password varchar, user_role varchar default 'client') as
$$
begin
    if not exists (select rolname from pg_roles where user_name = rolname) then
        insert into users values (user_name, user_password, user_role);
        execute 'create user "'||user_name||'" with login connection limit 1;';
    end if;
end;
$$ language plpgsql;




DROP PROCEDURE

CREATE PROCEDURE

Total execution time: 00:00:00.017

**Tests**

Total execution time: 00:00:00

**Triggres**

In [229]:
CREATE OR REPLACE TRIGGER contract_signed 
    AFTER UPDATE OF signed 
    ON raw_contracts
    FOR EACH ROW
    EXECUTE PROCEDURE archive_contract();

CREATE TRIGGER

Total execution time: 00:00:00.013

**Test**

In [230]:
delete from raw_contracts;
delete from archive;
delete from cars;
delete from parkings;
delete from clients;
delete from managers;

insert into parkings values 
(1, 'asfiaslfdgashdfguiapshg'),
(2, 'gflsdjiopsjdfoigaoijbdi');

insert into managers values 
('manager passport 1', 'manager name 1', 'manager phone 1', DEFAULT, 10, 'user2', 1),
('manager passport 2', 'manager name 2', 'manager phone 2', DEFAULT, 20, 'user4', 2);

insert into clients values 
('1234567891', 'client name', 'client phone', 'client email', now(), 'reg address', 'rev address', 'user1');

insert into cars values 
('123764192341192834912', 1.00, 'car brand 1', 'car color 1', 'car class 1', 'akpp', 'ai-95', 1),
('248597238945703124582', 2.00, 'car brand 2', 'car color 2', 'car class 2', 'mkpp', 'ai-95', 2),
('234587923452475245883', 3.00, 'car brand 3', 'car color 3', 'car class 3', 'akpp', 'ai-95', 2);

insert into raw_contracts values
(1, '1 day', 1000.00, '1234567891', '123764192341192834912');

insert into manager_on_contract values
('manager passport 1', 1);


select * from raw_contracts;
select * from archive;


update raw_contracts
set signed = true 
where ID_contract = 1;


select * from archive;
select * from raw_contracts;
select * from manager_on_contract;

DELETE 0

DELETE 0

DELETE 0

DELETE 0

DELETE 0

...

id_contract,rental_duration,total_price,driving_license,vin_number,signed
1,1 day,"$1,000.00",1234567891,123764192341192834912,False


id_record,id_contract,start_date,end_date,total_price,car_vin_number,client_driving_license,client_name,phone_number,manager_passport_details,manager_name


id_record,id_contract,start_date,end_date,total_price,car_vin_number,client_driving_license,client_name,phone_number,manager_passport_details,manager_name
1,1,2023-12-12,2023-12-13,"$1,000.00",123764192341192834912,1234567891,client name,client phone,manager passport 1,manager name 1


id_contract,rental_duration,total_price,driving_license,vin_number,signed


manager_passport_details,id_contract


**Roles**

In [231]:
/*
drop role if exists _admin;
drop role if exists _manager;
drop role if exists _client;


-- Create roles


create role _admin createrole login;
create role _manager inherit login;
create role _client inherit login;
*/

------------------------------------------

-- Admin configuration
grant all on all tables in schema public to _admin;


-- Manager configuration
grant select on cars, managers, archive to _manager;
grant insert on archive, accidents to _manager;



-- Client configuration
grant insert on raw_contracts to _client;
grant select on cars to _client;



------------------------------------------




-- Create admin users



-- Create manager users



-- Create client users





------------------------------------------


/*

create user user4;
grant _manager to user4;




grant select on cars, managers to _manager;
grant select, insert on archive, accidents to _manager;
grant update on raw_contracts to _manager;

grant insert on raw_contracts to _client;
*/


GRANT

GRANT

GRANT

GRANT

GRANT

Total execution time: 00:00:00.030

**Test**

In [232]:
set role postgres;

call add_user('new_user', 'new_password');

select * from users;

select rolname from pg_roles;

SET

CALL

(4 row(s) affected)

(20 row(s) affected)

Total execution time: 00:00:00.015

username,password,role
user1,password1,client
user2,password2,manager
user3,password3,admin
user4,password4,manager


rolname
pg_database_owner
pg_read_all_data
pg_write_all_data
pg_monitor
pg_read_all_settings
pg_read_all_stats
pg_stat_scan_tables
pg_read_server_files
pg_write_server_files
pg_execute_server_program


**Polices**

In [233]:
set role postgres;
alter table cars enable row level security;

-- drop policy manager_select_policy;
-- drop policy manager_select_cars_policy;

create policy manager_select_cars_policy on cars
    for select to _manager
    using ((select ID_parking from managers where username = current_user) = ID_parking);


SET

ALTER TABLE

CREATE POLICY

Total execution time: 00:00:00.021

**Test**

Total execution time: 00:00:00